# Detector de plágio em composição musical

### Referências

https://www.kaggle.com/code/eslamreda0101/plagirism-checker-using-scikit-learn-ml

https://www.kaggle.com/code/fanbyprinciple/detecting-plagiarism-in-text

In [1]:
import os
import glob

import numpy as np
import pandas as pd

from numpy import vectorize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
pd.set_option('max_colwidth', None)

## Efetuar leitura das letras musicais

In [154]:
#!tar xzf musicas.tgz
#!ls -R musicas

In [98]:
dir_raiz = "musicas"

In [99]:
bandas = os.listdir(dir_raiz)
bandas

['wesley-safadao', 'latino', 'rita-lee', 'pitty', 'mano-walter']

In [100]:
sample_files = glob.glob(dir_raiz + "/*/*.txt")
sample_contents = [open(File).read() for File in sample_files]

In [101]:
%%script true # comentar essa linha para fazer leitura parcial dos textos
qtd_limite = 1000
sample_files = sample_files[:qtd_limite]
len(sample_files)

1000

In [102]:
sample_files[:10]

['musicas/wesley-safadao/989577.txt',
 'musicas/wesley-safadao/depende-part-dj-guga-e-ze-felipe.txt',
 'musicas/wesley-safadao/parado-no-bailao.txt',
 'musicas/wesley-safadao/vou-dar-virote.txt',
 'musicas/wesley-safadao/saudade-nivel-hard.txt',
 'musicas/wesley-safadao/vamos-falar-de-amor-part-mara-pavanelly.txt',
 'musicas/wesley-safadao/vai-la.txt',
 'musicas/wesley-safadao/1837730.txt',
 'musicas/wesley-safadao/o-cara-errado.txt',
 'musicas/wesley-safadao/veja-so-no-que-deu.txt']

## Realizar vetorização dos textos

In [155]:
#vectorize = lambda text: TfidfVectorizer(stop_words=stopwords).fit_transform(text).toarray()
vectorize = lambda text: TfidfVectorizer().fit_transform(text).toarray()

In [156]:
similarity = lambda doc1, doc2: cosine_similarity([doc1, doc2])

In [157]:
vectors = vectorize(sample_contents)

In [158]:
s_vectors = list(zip(sample_files, vectors))

In [159]:
s_vectors[:2]

[('musicas/wesley-safadao/989577.txt', array([0., 0., 0., ..., 0., 0., 0.])),
 ('musicas/wesley-safadao/depende-part-dj-guga-e-ze-felipe.txt',
  array([0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.13661995]))]

## Calcular similaridades entre as letras musicais

In [116]:
%%time
pairs = set()
results = set()

# varrer as letras em um produto cartesiano
for sample_a, text_vector_a in s_vectors:
    for sample_b, text_vector_b in s_vectors:
        
        # ignorar se for o mesmo
        if sample_a == sample_b:
            continue
        
        sample_pair = sorted((sample_a, sample_b))
        pair_names = ';'.join(sample_pair)
        
        # ignorar se o par já tiver sido incluído
        if pair_names in pairs:
            continue
        pairs.add(pair_names)
        
        # calcular similaridade e incluir no resultado
        sim_score = similarity(text_vector_a, text_vector_b)[0][1]
        score = sample_pair[0], sample_pair[1], \
                np.count_nonzero(text_vector_a), \
                np.count_nonzero(text_vector_b), \
                sim_score
        results.add(score)

CPU times: user 8min 14s, sys: 3.3 s, total: 8min 18s
Wall time: 2min 4s


In [160]:
# gerar dataframe consolidado com as comparações

df = pd.DataFrame(results, columns=[
    'Musica1', 'Musica2', 'Tokens1', 'Tokens2', 'Similaridade']).round(3)
df['PercTokens'] = (df['Tokens1'] / df['Tokens2']).round(2)

obter_artista = lambda x: x.split('/')[1]
df['Artista1'] = df['Musica1'].apply(obter_artista)
df['Artista2'] = df['Musica2'].apply(obter_artista)

obter_musica = lambda x: x.split('/')[2].split('.txt')[0]
df['Musica1'] = df['Musica1'].apply(obter_musica)
df['Musica2'] = df['Musica2'].apply(obter_musica)

df = df[['Artista1', 'Artista2', 'Musica1', 'Musica2', 
         'Tokens1', 'Tokens2', 'PercTokens', 'Similaridade'
        ]].sort_values('Similaridade', ascending=False)

In [161]:
#!pip install -q pyarrow

In [162]:
# gravar o dataframe em formato binário
#df.to_parquet('df.parquet.gz', compression='gzip')

In [163]:
# ler dataframe previamente gerado
#df = pd.read_parquet('df.parquet.gz')

In [164]:
df

,Artista1,Artista2,Musica1,Musica2,Tokens1,Tokens2,PercTokens,Similaridade
239933,latino,wesley-safadao,1745377,festa-na-piscina,57,57,1.00,1.000
18886,wesley-safadao,wesley-safadao,2003669,ei-olha-o-som-empinadinha,33,30,1.10,0.999
402823,latino,latino,james-bom-de-cama-part-rionegro-e-solimoes,james-bom-de-cama,60,63,0.95,0.998
343800,latino,latino,831550,964806,34,36,0.94,0.994
119040,latino,latino,1743242,coelinha,54,58,0.93,0.994
...,...,...,...,...,...,...,...,...
378105,wesley-safadao,wesley-safadao,908112,lendas-e-misterios,67,19,3.53,0.000
378091,wesley-safadao,wesley-safadao,358920,bumbum-no-chao,27,25,1.08,0.000
92511,wesley-safadao,wesley-safadao,803624,bora,25,26,0.96,0.000
92513,latino,wesley-safadao,1038820,1600534,24,39,0.62,0.000


## Analisar comparações entre as letras

### Similaridade entre artistas distintos

In [165]:
df[(df['Artista1'] != df['Artista2']) & (df["Similaridade"] >= 0.5)].\
  groupby(['Artista1', 'Artista2']).\
  agg({'Similaridade': ['min', 'max', 'mean', 'std', 'median', 'count']}).\
  round(2)

Similaridade                               
                                 min   max  mean   std median count
Artista1 Artista2                                                  
latino   rita-lee               0.54  0.54  0.54   NaN   0.54     1
         wesley-safadao         0.50  1.00  0.72  0.18   0.64    24
rita-lee wesley-safadao         0.50  0.60  0.55  0.07   0.55     2

### Similaridade no mesmo artista

In [166]:
df[(df['Artista1'] == df['Artista2'])].\
  groupby(['Artista1']).\
  agg({'Similaridade': ['min', 'max', 'mean', 'std', 'median', 'count']}).\
  round(2)

Similaridade                                 
                        min   max  mean   std median   count
Artista1                                                    
latino                  0.0  1.00  0.02  0.04   0.01   14196
rita-lee                0.0  0.15  0.01  0.02   0.00     528
wesley-safadao          0.0  1.00  0.03  0.03   0.02  318003

### Letras musicais mais similares

In [167]:
df[(df['Similaridade'] < 0.95) & (df['Similaridade'] > 0.7)].head(10)

,Artista1,Artista2,Musica1,Musica2,Tokens1,Tokens2,PercTokens,Similaridade
141012,wesley-safadao,wesley-safadao,1106034,beber-e-paquerar-part-rai-saia-rodada,27,30,0.90,0.947
256695,latino,latino,1816211,beber-deitado,35,34,1.03,0.946
224465,latino,wesley-safadao,1956598,1973034,58,61,0.95,0.944
260542,wesley-safadao,wesley-safadao,despedida,na-despedida,50,52,0.96,0.944
391772,latino,latino,101927,me-leva-2017-ao-vivo-part-caio-giovani,89,40,2.22,0.934
111041,latino,latino,205187,mel-da-sua-boca,40,42,0.95,0.931
213559,latino,latino,1278158,amor-de-estudante,61,60,1.02,0.927
255573,latino,wesley-safadao,festa-na-piscina,festa-na-piscina,57,62,0.92,0.916
179610,latino,latino,1745377,festa-na-piscina,57,62,0.92,0.916
342,wesley-safadao,wesley-safadao,1616936,920111,26,28,0.93,0.913


In [168]:
df[(df['Similaridade'] < 0.8) & (df['Similaridade'] > 0.6)].head(10)

,Artista1,Artista2,Musica1,Musica2,Tokens1,Tokens2,PercTokens,Similaridade
185729,latino,wesley-safadao,831550,hoje-e-dia,50,36,1.39,0.798
177118,latino,wesley-safadao,964806,hoje-e-dia,50,34,1.47,0.767
15540,wesley-safadao,wesley-safadao,1856104,1862510,57,74,0.77,0.764
426765,wesley-safadao,wesley-safadao,1210174,assim-voce-mata-o-papai,38,33,1.15,0.747
11722,wesley-safadao,wesley-safadao,1691494,pout-pourri-so-freud-explica-meu-amanhecer-ninguem-vai-separar,103,25,4.12,0.747
370704,wesley-safadao,wesley-safadao,escravo-do-amor-tentativas-em-vao-meu-amanhecer-menino-bobo-pot-pourri,tentativas-em-vao-part-marcia-fellipe,161,40,4.03,0.741
279964,wesley-safadao,wesley-safadao,1862473,abertura-garota-vip,53,74,0.72,0.738
341039,wesley-safadao,wesley-safadao,1756934,1757583,56,49,1.14,0.724
408130,wesley-safadao,wesley-safadao,1595953,escravo-do-amor-tentativas-em-vao-meu-amanhecer-menino-bobo-pot-pourri,39,161,0.24,0.722
390269,wesley-safadao,wesley-safadao,307314,358918,37,58,0.64,0.721


## Avaliação de exemplos

In [169]:
idpar = 224465

#idpar = 784 # lança perfume
#idpar = 65078  # agora é pra valer
#idpar = 674960 # agora só falta você

In [170]:
par = df.loc[idpar]
musica1 = "%s/%s/%s.txt" % (dir_raiz, par[0], par[2])
musica2 = "%s/%s/%s.txt" % (dir_raiz, par[1], par[3])
par #, musica1, musica2

Artista1                latino
Artista2        wesley-safadao
Musica1                1956598
Musica2                1973034
Tokens1                     58
Tokens2                     61
PercTokens                0.95
Similaridade             0.944
Name: 224465, dtype: object

In [171]:
letra1 = sample_contents[sample_files.index(musica1)]
letra2 = sample_contents[sample_files.index(musica2)]

In [172]:
letra1

'Dança Kuduro (part. Daddy Kall)\n\nDança Kuduro! Vem Latino, Daddy Kall\nA mão pra cima, cintura solta Da meia volta, dança Kuduro Não se canse agora começou a festa Mexe a cabeça e dança Kuduro\nA mão pra cima, cintura solta Da meia volta, dança Kuduro Não se canse agora começou a festa Mexe a cabeça e dança Kuduro\nQuem pode domar a força Que entra nas suas veias? Fica quente gruda na gente Ferve esquenta, incendeia\nE quem pode parar isso aqui Descontrola sua cadeira Esse fogo que queima por dentro E lento tudo me tempera\nA mão pra cima, cintura solta Da meia volta e sacode duro Não se canse agora começou a festa Mexe a cabeça e sacode duro\nBalança que é uma loucura Morena vem do meu lado Ninguém vai ficar parado Quero ver mexer Kuduro\nBalança que é uma loucura Morena vem do meu lado Ninguém vai ficar parado\nOi, oi, oi, oi, oi, oi, oi? É para quebrar Kuduro, vamos dançar Kuduro Oi, oi, oi, oi, oi, oi, oi? Seja morena ou loira, vem balançar Kuduro Oi, oi, oi?\nA mão pra cima, ci

In [173]:
letra2

'Dança Kuduro\n\nA mão pra cima, a cintura solta Da meia volta, dança Kuduro Não se canse agora começou a festa Mexe a cabeça e dança Kuduro (2x)\nQuem pode domar a força Que entra nas suas veias? Fica quente gruda na gente Ferve esquenta, incendeia\nE quem pode parar isso aqui Descontrola sua cadeira Esse fogo que queima por dentro E lento tudo me tempera\nA mão pra cima, a cintura solta Da meia volta e sacode duro Não se canse agora começou a festa Mexe a cabeça e sacode duro\nBalança que é uma loucura Morena vem ao meu lado Ninguém vai ficar parado Quero ver mexer kuduro\nBalança que é uma loucura Morena vem ao meu lado Ninguém vai ficar parado\nOi, oi, oi, oi, oi, oi, oi Vem para quebrar kuduro, vamos dançar kuduro Oi, oi, oi, oi, oi, oi, oi Seja morena ou loira, vem balançar kuduro Oi, oi, oi\n\n'

## Avaliação de dois exemplos

In [174]:
#!pip install -U nltk

In [175]:
import nltk
#nltk.download('punkt')
#nltk.download('stopwords')

In [176]:
import re
from nltk.util import ngrams, pad_sequence, everygrams
from nltk.tokenize import word_tokenize
from nltk.lm import MLE, WittenBellInterpolated
import numpy as np
import plotly.graph_objects as go
from scipy.ndimage import gaussian_filter

In [178]:
stopwords = nltk.corpus.stopwords.words('portuguese')
stopwords[:10]

['a',
 'à',
 'ao',
 'aos',
 'aquela',
 'aquelas',
 'aquele',
 'aqueles',
 'aquilo',
 'as']

In [179]:
# Training data file
train_data_file = musica1 #"train.txt"

# read training dataz
with open(train_data_file) as f:
    train_text = f.read().lower()

# apply preprocessing (remove text inside square and curly brackets and rem punc)
train_text = re.sub(r"\[.*\]|\{.*\}", "", train_text)
train_text = re.sub(r'[^\w\s]', "", train_text)
train_text = re.sub(r'[\n]', " ", train_text)

In [180]:
train_text

'dança kuduro part daddy kall  dança kuduro vem latino daddy kall a mão pra cima cintura solta da meia volta dança kuduro não se canse agora começou a festa mexe a cabeça e dança kuduro a mão pra cima cintura solta da meia volta dança kuduro não se canse agora começou a festa mexe a cabeça e dança kuduro quem pode domar a força que entra nas suas veias fica quente gruda na gente ferve esquenta incendeia e quem pode parar isso aqui descontrola sua cadeira esse fogo que queima por dentro e lento tudo me tempera a mão pra cima cintura solta da meia volta e sacode duro não se canse agora começou a festa mexe a cabeça e sacode duro balança que é uma loucura morena vem do meu lado ninguém vai ficar parado quero ver mexer kuduro balança que é uma loucura morena vem do meu lado ninguém vai ficar parado oi oi oi oi oi oi oi é para quebrar kuduro vamos dançar kuduro oi oi oi oi oi oi oi seja morena ou loira vem balançar kuduro oi oi oi a mão pra cima cintura solta da meia volta dança kuduro não 

In [181]:
# set ngram number
n = 4

# pad the text and tokenize
training_data = list(pad_sequence(word_tokenize(train_text, language="portuguese"), n, 
                                  pad_left=True, 
                                  left_pad_symbol="<s>"))

# generate ngrams
ngrams = list(everygrams(training_data, max_len=n))
print("Number of ngrams:", len(ngrams))

# build ngram language models
model = WittenBellInterpolated(n)
model.fit([ngrams], vocabulary_text=training_data)
print(model.vocab)

Number of ngrams: 1398
<Vocabulary with cutoff=1 unk_label='<UNK>' and 85 items>


In [182]:
# testing data file
test_data_file = musica2 #"test.txt"

# Read testing data
with open(test_data_file) as f:
    test_text = f.read().lower()
    
test_text = re.sub(r'[^\w\s]', "", test_text)
test_text = re.sub(r'[\n]', " ", test_text)

In [183]:
test_text

'dança kuduro  a mão pra cima a cintura solta da meia volta dança kuduro não se canse agora começou a festa mexe a cabeça e dança kuduro 2x quem pode domar a força que entra nas suas veias fica quente gruda na gente ferve esquenta incendeia e quem pode parar isso aqui descontrola sua cadeira esse fogo que queima por dentro e lento tudo me tempera a mão pra cima a cintura solta da meia volta e sacode duro não se canse agora começou a festa mexe a cabeça e sacode duro balança que é uma loucura morena vem ao meu lado ninguém vai ficar parado quero ver mexer kuduro balança que é uma loucura morena vem ao meu lado ninguém vai ficar parado oi oi oi oi oi oi oi vem para quebrar kuduro vamos dançar kuduro oi oi oi oi oi oi oi seja morena ou loira vem balançar kuduro oi oi oi  '

In [184]:
# Tokenize and pad the text
testing_data = list(pad_sequence(word_tokenize(test_text, language="portuguese"), n, 
                                 pad_left=True,
                                 left_pad_symbol="<s>"))
print("Length of test data:", len(testing_data))

Length of test data: 158


In [185]:
# assign scores
scores = []
for i, item in enumerate(testing_data[n-1:]):
    s = model.score(item, testing_data[i:i+n-1])
    scores.append(s)

scores_np = np.array(scores)

In [186]:
# set width and height
width = 8
height = np.ceil(len(testing_data)/width).astype("int32")
print("Width, Height:", width, ",", height)

# copy scores to rectangular blank array
a = np.zeros(width*height)
a[:len(scores_np)] = scores_np
diff = len(a) - len(scores_np)

# apply gaussian smoothing for aesthetics
a = gaussian_filter(a, sigma=1.0)

# reshape to fit rectangle
a = a.reshape(-1, width)

# format labels
labels = [" ".join(testing_data[i:i+width]) for i in range(n-1, len(testing_data), width)]
labels_individual = [x.split() for x in labels]
labels_individual[-1] += [""]*diff
labels = [f"{x:60.60}" for x in labels]

Width, Height: 8 , 20


In [187]:
# create heatmap
from matplotlib import pyplot as plt
fig = go.Figure(data=go.Heatmap(
                z=a, x0=0, dx=1,
                y=labels, zmin=0, zmax=1,
                customdata=labels_individual,
                hovertemplate='%{customdata} <br><b>Score:%{z:.3f}<extra></extra>',
                colorscale="burg"))
fig.update_layout({"height":height*28, "width":1000, "font":{"family":"Courier New"}})
fig['layout']['yaxis']['autorange'] = "reversed"
fig.show()
#fig.save_fig("heatmap.png")